<a href="https://www.kaggle.com/code/avikumart/healthcare-datasci-retinal-fundus-segmentation?scriptVersionId=266880689" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd 
import os
import numpy as np
from PIL import Image
import cv2
from scipy.ndimage import binary_opening
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization, ReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

2025-10-09 17:00:09.757636: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760029210.165022      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760029210.285970      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images/34_training.tif
/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images/40_training.tif
/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images/22_training.tif
/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images/27_training.tif
/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images/28_training.tif
/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images/26_training.tif
/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images/38_training.tif
/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images/39_training.tif
/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images/24_training.tif
/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/tr

In [3]:
# datasets path

# Dataset paths
train_images_path = "/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images"
train_manual_path = "/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/1st_manual"
train_mask_path = "/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/mask"
test_images_path = "/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/test/images"
test_mask_path = "/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/test/mask"

def check_dir(path):
    if os.path.exists(path):
        print(f"{path} exists")
    else:
        print(f"{path} does not exist")
    return os.path.exists(path)

# check the dirs
check_dir(train_images_path)
check_dir(test_images_path)

/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images exists
/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/test/images exists


True

In [4]:
# load the images
def load_images(img_path):
    img = Image.open(img_path)
    if img.mode == "RGB":
        img = np.array(img)
        green_channel = img[:,:,1]
        return green_channel
    else:
        return np.array(img)

In [5]:
image = load_images("/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/test/images/02_test.tif")
image.shape

(584, 565)

In [6]:
# pre-process the images
def preprocess_image(image):
    mean = np.mean(image)
    std = np.std(image) if np.std(image) != 0 else 1
    gcn_image = (image- mean)/std
    gcn_image = np.clip(gcn_image, 0, 255).astype(np.uint8)
    clahe = cv2.createCLAHE(clipLimit = 2.0, tileGridSize=(8,8))
    clahe_image = clahe.apply(gcn_image)
    gamma = 1.2
    gamma_image = np.array(255 * (clahe_image / 255) ** (1.0 / gamma), dtype=np.uint8)
    return gamma_image

In [7]:
def extract_patches(image, mask, manual=None, patch_size=64):
    patches =  []
    labels = []
    height, width = image.shape
    step = patch_size // 4
    pad_size = patch_size // 2
    padded_image = np.pad(image, ((pad_size,pad_size),(pad_size, pad_size)), mode="constant")
    padded_mask = np.pad(mask, ((pad_size, pad_size), (pad_size, pad_size)), mode="constant")
    padded_manual = np.pad(manual, ((pad_size, pad_size), (pad_size, pad_size)), mode="constant")

    # loop over the image to extract the patches
    for i  in range(pad_size, height + pad_size, step):
        for j in range(pad_size, width + pad_size, step):
            if padded_mask[i,j] == 0:
                continue
            patch = padded_image[i - pad_size:i+pad_size, j - pad_size:j +pad_size]
            if patch.shape != (patch_size, patch_size):
                continue
            if manual is not None:
                label = padded_manual[i, j] // 255
                labels.append(label)
            patches.append(patch)


    patches = np.array(patches)
    labels = np.array(labels) if manual is not None else None
    if manual is not None:
        print(f"Extracted {len(patches)} patches: {np.sum(labels == 1)} vessel, {np.sum(labels == 0)} non-vessel")
    else:
        print(f"extracted {len(patches)} patches for prediction")
    return patches, labels

In [7]:
# level-I balancing
def level_i_balancing(patches, labels, threshold=0.1, rho=0.7):
    patch_means = np.mean(patches, axis=(1,2)) / 255.0
    vessel_patches = patches[labels == 1]
    non_vessel_patches = patches[labels == 0]
    non_vessel_means = patch_means[labels == 0]

    partial_background = non_vessel_patches[non_vessel_means >= threshold]
    full_background = non_vessel_patches[non_vessel_means < threshold]

    num_vessel = len(vessel_patches)
    num_partial = int(num_vessel + rho)
    num_full = int(num_vessel * (1 - rho))
    # initialize the empty array
    empty_patch_array = np.empty((0,64,64))
    partial_indices = np.random.choice(len(partial_background), min(num_partial), len(partial_background)), replace=False if len(partial_background) > 0 else []
    full_indices = np.random.choice(len(full_background), min(num_full, len(full_background)), replace=False if len(full_background) > 0 else []

    selected_partial = partial_background[partial_indices] if len(partial_indices) > 0 else empty_patch_array
    selected_full = full_background[full_indices] if len(full_indices) > 0 else empty_patch_array

    # concate only non-empty arrays:
    arrays_to_concat = [vessel_patches]
    if selected_partial.shape[0] > 0:
        arrays_to_concat.append(selecte_partial)
    if selected_full.shape[0] > 0:
        arrays_to_concat.append(selected_full)
        
    balanced_patches = np.concatenate(arrays_to_concat, axis=0) if len(arrays_to_concat) > 1 else vessel_patches
    balanced_labels = np.concatenate([
        np.ones(len(vessel_patches)),
        np.zeros(len(selected_partial) + len(selected_full))
    ]) if len(arrays_to_concat) > 1 else np.ones(len(vessel_patches))
    
    print(f"Balanced dataset: {len(vessel_patches)} vessel, {len(selected_partial)} partial background, {len(selected_full)} full background")
    return balanced_patches, balanced_labels

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (174495916.py, line 16)

In [8]:
# build BLCB-CNN model for the segmentation
def build_blcb_cnn(input_shape=(64,64,1)):
    model = Sequential([
        Input(shape=input_shape),
        Conv2D(32, (3,3), padding="same"),
        BatchNormalization(),
        ReLU(),
        Conv2D(32, (3,3), padding="same"),
        BatchNormalization(),
        ReLU(),
        MaxPooling2D((2,2)),
        Dropout(0.20),
        Conv2D(64, (3, 3), padding='same'),
        BatchNormalization(),
        ReLU(),
        Conv2D(64, (3, 3), padding='same'),
        BatchNormalization(),
        ReLU(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Conv2D(128, (3, 3), padding='same'),
        BatchNormalization(),
        ReLU(),
        Conv2D(128, (3, 3), padding='same'),
        BatchNormalization(),
        ReLU(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Flatten(),
        Dense(512),
        BatchNormalization(),
        ReLU(),
        Dropout(0.25),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [9]:
def postprocess_mask(pred_mask):
    kernel = np.ones((3, 3), np.uint8)
    eroded_mask = cv2.erode(pred_mask, kernel, iterations=1)
    return eroded_mask

In [10]:
# Predict Segmentation Mask for an Image
def predict_mask(model, image, mask, patch_size=64):
    preprocessed_image = preprocess_image(image) * (mask / 255)
    patches, _ = extract_patches(preprocessed_image, mask, manual=None)
    patches = patches.reshape(-1, 64, 64, 1)
    pred_patches = model.predict(patches, verbose=0)
    pred_patches = (pred_patches > 0.3).astype(np.uint8).flatten()
    
    height, width = image.shape
    pred_mask = np.zeros((height, width), dtype=np.float32)
    count_mask = np.zeros((height, width), dtype=np.float32)
    step = patch_size // 4
    idx = 0
    for i in range(0, height, step):
        for j in range(0, width, step):
            if i >= height or j >= width or mask[i, j] == 0:
                continue
            if idx < len(pred_patches):
                pred_mask[i, j] += pred_patches[idx]
                count_mask[i, j] += 1
                idx += 1
    
    pred_mask = np.where(count_mask > 0, pred_mask / count_mask, 0) * 255
    pred_mask = pred_mask.astype(np.uint8)
    print(f"Processed {idx} patches for prediction, Predicted mask: {np.sum(pred_mask > 0)} vessel pixels")
    return postprocess_mask(pred_mask)